In [2]:
import face_recognition
import cv2
import pickle
import numpy as np
from scipy.spatial import distance as dist

# Configurations
THRESHOLD = 0.45
UNKNOWN_NAME = "Unknown"
EYE_AR_THRESH = 0.25  # Eye Aspect Ratio threshold
EYE_AR_CONSEC_FRAMES = 3  # Frames to confirm blink
REQUIRED_BLINKS = 2  # Minimum blinks to verify liveness
HEAD_MOVEMENT_THRESH = 15  # Pixel movement threshold for head motion

def load_known_faces(filename):
    """Load known face encodings and names from file"""
    try:
        with open(filename, "rb") as f:
            known_face_encodings, known_face_names = pickle.load(f)
        return known_face_encodings, known_face_names
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found. Ensure it exists.")
        return [], []

def eye_aspect_ratio(eye):
    """Compute Eye Aspect Ratio (EAR) for blink detection"""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def is_real_face(landmarks, prev_face_pos):
    """Check liveness using blink + head movement + texture"""
    # 1. Blink Detection
    left_eye = landmarks["left_eye"]
    right_eye = landmarks["right_eye"]
    left_ear = eye_aspect_ratio(left_eye)
    right_ear = eye_aspect_ratio(right_eye)
    ear = (left_ear + right_ear) / 2.0
    is_blinking = ear < EYE_AR_THRESH

    # 2. Head Movement (Compare nose position)
    nose = landmarks["nose_tip"][0]
    if prev_face_pos:
        movement = dist.euclidean(nose, prev_face_pos)
        has_moved = movement > HEAD_MOVEMENT_THRESH
    else:
        has_moved = False

    # 3. Texture Analysis (Brightness variance)
    # (Note: Requires face_roi, but we don't have it here. Skip or add later.)
    is_live = is_blinking or has_moved
    return is_live, nose

def recognize_faces(frame, known_encodings, known_names, prev_face_pos, liveness_counter):
    """Recognize faces with liveness checks"""
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    face_landmarks_list = face_recognition.face_landmarks(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding, landmarks in zip(
        face_locations, face_encodings, face_landmarks_list
    ):
        # Liveness Check
        is_live, nose_pos = is_real_face(landmarks, prev_face_pos)
        if is_live:
            liveness_counter += 1
            prev_face_pos = nose_pos

        # Only recognize after liveness confirmation
        if liveness_counter >= REQUIRED_BLINKS:
            face_distances = face_recognition.face_distance(known_encodings, face_encoding)
            best_match_idx = np.argmin(face_distances)
            best_distance = face_distances[best_match_idx]
            name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
            color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)
        else:
            name = "Move/Blink to verify"
            color = (255, 255, 0)  # Yellow (pending)

        # Draw bounding box
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return frame, prev_face_pos, liveness_counter

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    if not known_encodings:
        return  # Exit if no known faces loaded

    video_capture = cv2.VideoCapture(0)
    prev_face_pos = None
    liveness_counter = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame, prev_face_pos, liveness_counter = recognize_faces(
            frame, known_encodings, known_names, prev_face_pos, liveness_counter
        )
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [1]:
import face_recognition
import cv2
import pickle
import numpy as np
import os
from datetime import datetime

# Configuration
THRESHOLD = 0.45  # Similarity threshold
UNKNOWN_NAME = "Unknown"
MIN_FACE_SIZE = 100  # Minimum pixel size for a face to be considered
BLUR_THRESHOLD = 100  # Threshold for blur detection (lower is more blurry)
SCREEN_REFLECTION_THRESHOLD = 200  # Brightness threshold for screen detection

def load_known_faces(filename):
    """Load known face encodings and names from file with error handling"""
    try:
        with open(filename, "rb") as f:
            known_face_encodings, known_face_names = pickle.load(f)
        return known_face_encodings, known_face_names
    except (FileNotFoundError, EOFError, pickle.UnpicklingError) as e:
        print(f"Error loading face database: {e}")
        return [], []

def is_screen_face(frame, face_location):
    """Check if the face is likely on a screen (photo/video)"""
    (top, right, bottom, left) = face_location
    face_region = frame[top:bottom, left:right]
    gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray_face)
    return avg_brightness > SCREEN_REFLECTION_THRESHOLD

def is_too_blurry(frame):
    """Check if the frame is too blurry for reliable detection"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var() < BLUR_THRESHOLD

def is_face_too_small(face_location):
    """Check if the detected face is too small to be real"""
    (top, right, bottom, left) = face_location
    return (right - left) < MIN_FACE_SIZE or (bottom - top) < MIN_FACE_SIZE

def recognize_faces(frame, known_encodings, known_names):
    """Enhanced face recognition with filtering"""
    if is_too_blurry(frame):
        cv2.putText(frame, "Low quality - ignoring", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        return frame

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    
    for face_location in face_locations:
        if is_face_too_small(face_location) or is_screen_face(frame, face_location):
            continue
            
        face_encoding = face_recognition.face_encodings(rgb_frame, [face_location])[0]
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_idx = np.argmin(face_distances)
        best_distance = face_distances[best_match_idx]
        
        name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
        (top, right, bottom, left) = face_location
        
        # Draw rectangle and label
        color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
        
        # Display confidence score
        confidence = 1 - best_distance
        cv2.putText(frame, f"{confidence:.2f}", (left, bottom + 45), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    return frame

def log_attendance(name):
    """Log recognized faces with timestamp"""
    if name != UNKNOWN_NAME:
        with open("attendance.csv", "a") as f:
            f.write(f"{name},{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    
    if not known_encodings:
        print("No face database found. Please create one first.")
        return
    
    video_capture = cv2.VideoCapture(0)
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
            
        frame = recognize_faces(frame, known_encodings, known_names)
        
        # # Display FPS
        # cv2.putText(frame, f"FPS: {int(video_capture.get(cv2.CAP_PROP_FPS))}", 
        #            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Face Recognition', frame)
        
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('a'):  # Add new face to database
            add_new_face(frame, known_encodings, known_names)
    
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [4]:
import face_recognition
import cv2
import pickle
import numpy as np
import os
from datetime import datetime

# Configuration
THRESHOLD = 0.4  # Similarity threshold
UNKNOWN_NAME = "Unknown"
MIN_FACE_SIZE = 100  # Minimum pixel size for a face to be considered
BLUR_THRESHOLD = 100  # Threshold for blur detection (lower is more blurry)
SCREEN_REFLECTION_THRESHOLD = 200  # Brightness threshold for screen detection
FRAME_SKIP = 1   # Process every nth frame to reduce load
RESIZE_FACTOR = 0.5  # Resize frame for faster processing (0.5 = half size)

def load_known_faces(filename):
    """Load known face encodings and names from file with error handling"""
    try:
        with open(filename, "rb") as f:
            known_face_encodings, known_face_names = pickle.load(f)
        return known_face_encodings, known_face_names
    except (FileNotFoundError, EOFError, pickle.UnpicklingError) as e:
        print(f"Error loading face database: {e}")
        return [], []

def is_screen_face(frame, face_location):
    """Check if the face is likely on a screen (photo/video)"""
    (top, right, bottom, left) = face_location
    face_region = frame[top:bottom, left:right]
    if face_region.size == 0:  # Skip if region is empty
        return False
    gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray_face)
    return avg_brightness > SCREEN_REFLECTION_THRESHOLD

def is_too_blurry(frame):
    """Check if the frame is too blurry for reliable detection"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var() < BLUR_THRESHOLD

def is_face_too_small(face_location):
    """Check if the detected face is too small to be real"""
    (top, right, bottom, left) = face_location
    return (right - left) < MIN_FACE_SIZE or (bottom - top) < MIN_FACE_SIZE

def recognize_faces(frame, known_encodings, known_names, frame_counter):
    """Optimized face recognition with filtering"""
    # Skip processing for some frames to improve performance
    if frame_counter % FRAME_SKIP != 0:
        return frame, []
    
    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=RESIZE_FACTOR, fy=RESIZE_FACTOR)
    
    if is_too_blurry(small_frame):
        cv2.putText(frame, "Low quality - ignoring", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        return frame, []

    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_small_frame)
    
    recognized_names = []
    
    for face_location in face_locations:
        # Scale back up face locations since the frame was resized
        top, right, bottom, left = [int(coord / RESIZE_FACTOR) for coord in face_location]
        
        if is_face_too_small((top, right, bottom, left)) or is_screen_face(frame, (top, right, bottom, left)):
            continue
            
        face_encoding = face_recognition.face_encodings(
            rgb_small_frame, 
            [face_location]
        )[0]
        
        # Only compare with known faces if we have any
        if known_encodings:
            face_distances = face_recognition.face_distance(known_encodings, face_encoding)
            best_match_idx = np.argmin(face_distances)
            best_distance = face_distances[best_match_idx]
            name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
        else:
            name = UNKNOWN_NAME
        
        recognized_names.append(name)
        
        # Draw rectangle and label on original frame
        color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
        
        # Display confidence score if we have known faces
        if known_encodings:
            confidence = 1 - best_distance
            cv2.putText(frame, f"{confidence:.2f}", (left, bottom + 45), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    return frame, recognized_names

def log_attendance(name):
    """Log recognized faces with timestamp"""
    if name != UNKNOWN_NAME:
        with open("attendance.csv", "a") as f:
            f.write(f"{name},{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    
    if not known_encodings:
        print("No face database found. Please create one first.")
        return
    
    video_capture = cv2.VideoCapture(0)
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    # Try to enable hardware acceleration if available
    try:
        video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
    except:
        pass
    
    frame_counter = 0
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
            
        frame_counter += 1
        processed_frame, recognized_names = recognize_faces(frame, known_encodings, known_names, frame_counter)
        
        # Log attendance for recognized faces
        for name in recognized_names:
            log_attendance(name)
        
        # Display FPS
        fps = video_capture.get(cv2.CAP_PROP_FPS)
        cv2.putText(processed_frame, f"FPS: {fps:.1f}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Face Recognition', processed_frame)
        
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [5]:
import face_recognition
import cv2
import pickle
import numpy as np
import os
from datetime import datetime

# Configuration
THRESHOLD = 0.4  # Similarity threshold
UNKNOWN_NAME = "Unknown"
MIN_FACE_SIZE = 100  # Minimum pixel size for a face to be considered
BLUR_THRESHOLD = 100  # Threshold for blur detection (lower is more blurry)
SCREEN_REFLECTION_THRESHOLD = 200  # Brightness threshold for screen detection
EDGE_VARIATION_THRESHOLD = 20  # Threshold for edge variation detection (screen faces tend to have less variation)

def load_known_faces(filename):
    """Load known face encodings and names from file with error handling"""
    try:
        with open(filename, "rb") as f:
            known_face_encodings, known_face_names = pickle.load(f)
        return known_face_encodings, known_face_names
    except (FileNotFoundError, EOFError, pickle.UnpicklingError) as e:
        print(f"Error loading face database: {e}")
        return [], []

def is_screen_face(frame, face_location):
    """Enhanced check if the face is likely on a screen (photo/video)"""
    (top, right, bottom, left) = face_location
    face_region = frame[top:bottom, left:right]
    
    # Check brightness
    gray_face = cv2.cvtColor(face_region, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray_face)
    
    # Check edge variation (real faces have more complex edges)
    edges = cv2.Canny(gray_face, 100, 200)
    edge_variation = np.std(edges)
    
    # Check color saturation (screen faces often have lower saturation)
    hsv = cv2.cvtColor(face_region, cv2.COLOR_BGR2HSV)
    avg_saturation = np.mean(hsv[:,:,1])
    
    # Combine multiple factors
    brightness_factor = avg_brightness > SCREEN_REFLECTION_THRESHOLD
    edge_factor = edge_variation < EDGE_VARIATION_THRESHOLD
    saturation_factor = avg_saturation < 50  # Lower saturation threshold
    
    # If at least two factors indicate a screen face
    return (brightness_factor + edge_factor + saturation_factor) >= 2

def is_too_blurry(frame):
    """Check if the frame is too blurry for reliable detection"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var() < BLUR_THRESHOLD

def is_face_too_small(face_location):
    """Check if the detected face is too small to be real"""
    (top, right, bottom, left) = face_location
    return (right - left) < MIN_FACE_SIZE or (bottom - top) < MIN_FACE_SIZE

def recognize_faces(frame, known_encodings, known_names):
    """Enhanced face recognition with filtering"""
    if is_too_blurry(frame):
        cv2.putText(frame, "Low quality - ignoring", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        return frame

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    
    for face_location in face_locations:
        if is_face_too_small(face_location):
            continue
            
        if is_screen_face(frame, face_location):
            (top, right, bottom, left) = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
            cv2.putText(frame, "Screen Face Detected", (left, bottom + 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1)
            continue
            
        face_encoding = face_recognition.face_encodings(rgb_frame, [face_location])[0]
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_idx = np.argmin(face_distances)
        best_distance = face_distances[best_match_idx]
        
        name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
        (top, right, bottom, left) = face_location
        
        # Draw rectangle and label
        color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
        
        # Display confidence score
        confidence = 1 - best_distance
        cv2.putText(frame, f"{confidence:.2f}", (left, bottom + 45), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
        
        log_attendance(name)
    
    return frame

def log_attendance(name):
    """Log recognized faces with timestamp"""
    if name != UNKNOWN_NAME:
        with open("attendance.csv", "a") as f:
            f.write(f"{name},{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    
    if not known_encodings:
        print("No face database found. Please create one first.")
        return
    
    video_capture = cv2.VideoCapture(0)
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
            
        frame = recognize_faces(frame, known_encodings, known_names)
        
        cv2.imshow('Face Recognition', frame)
        
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [6]:
import face_recognition
import cv2
import pickle
import numpy as np
import random
import time

# Configuration
THRESHOLD = 0.6  # Increased threshold for better accuracy with CNN
UNKNOWN_NAME = "Unknown"
CHALLENGE_DURATION = 10
PENALTY_DURATION = 15
CHALLENGE_COOLDOWN = 10
USE_CNN_MODEL = True  # Using CNN model
DETECTION_SCALE = 0.25  # Scale for face detection (keeps encoding at full resolution)
PROCESS_EVERY_N_FRAMES = 1  # Process every frame for best accuracy

def load_known_faces(filename):
    """Load known face encodings and names from file"""
    try:
        with open(filename, "rb") as f:
            known_face_encodings, known_face_names = pickle.load(f)
        print(f"Loaded {len(known_face_encodings)} known faces")
        return known_face_encodings, known_face_names
    except Exception as e:
        print(f"Error loading known faces: {e}")
        return [], []

def detect_faces_cnn(frame):
    """Detect faces using CNN model with proper scaling"""
    # Convert to RGB (face_recognition uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Resize frame for faster detection while maintaining aspect ratio
    small_frame = cv2.resize(rgb_frame, (0, 0), fx=DETECTION_SCALE, fy=DETECTION_SCALE)
    
    # Find face locations with CNN model
    face_locations = face_recognition.face_locations(
        small_frame,
        model="cnn",
        number_of_times_to_upsample=1  # Reduce upsampling for speed
    )
    
    # Scale face locations back to original frame size
    return [
        (int(top/DETECTION_SCALE), int(right/DETECTION_SCALE), 
        int(bottom/DETECTION_SCALE), int(left/DETECTION_SCALE))
        for (top, right, bottom, left) in face_locations
    ], rgb_frame

def recognize_faces(frame, known_encodings, known_names, challenge_status, penalty_list):
    """Recognize faces with proper CNN implementation"""
    current_time = time.time()
    
    # Detect faces using CNN
    face_locations, rgb_frame = detect_faces_cnn(frame)
    
    # Get face encodings from the original resolution
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Check if face is in penalty
        in_penalty = False
        for penalty in penalty_list:
            if current_time - penalty['time'] < PENALTY_DURATION:
                distance = face_recognition.face_distance([penalty['encoding']], face_encoding)[0]
                if distance < 0.5:
                    in_penalty = True
                    break
        
        if in_penalty:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, "Unknown (Penalty)", (left, bottom + 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            continue
        
        # Find matches from known faces
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=THRESHOLD)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        
        best_match_idx = np.argmin(face_distances)
        best_distance = face_distances[best_match_idx]
        
        if matches[best_match_idx] and best_distance <= THRESHOLD:
            name = known_names[best_match_idx]
            color = (0, 255, 0)  # Green
            
            # Challenge system logic
            needs_challenge = (
                name not in challenge_status or 
                (not challenge_status[name]['active'] and 
                 current_time - challenge_status[name]['last_challenge'] > CHALLENGE_COOLDOWN)
            )
            
            if needs_challenge:
                challenge_status[name] = {
                    'active': True,
                    'text': get_challenge(),
                    'start_time': current_time,
                    'last_challenge': current_time,
                    'prev_frame': frame.copy(),
                    'encoding': face_encoding
                }
        else:
            name = UNKNOWN_NAME
            color = (0, 0, 255)  # Red
        
        # Draw rectangle and label
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    return frame, challenge_status

def main():
    # Load known faces
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    if not known_encodings:
        print("No known faces loaded. Please check your face_encodings.pkl file")
        return
    
    # Initialize video capture
    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Error: Could not open video source")
        return
    
    challenge_status = {}
    penalty_list = []
    frame_counter = 0
    
    print(f"Starting face recognition with CNN model (Threshold: {THRESHOLD})")
    
    try:
        while True:
            ret, frame = video_capture.read()
            if not ret:
                print("Error: Could not read frame")
                break
            
            frame_counter += 1
            if frame_counter % PROCESS_EVERY_N_FRAMES != 0:
                continue
            
            # Process frame
            processed_frame, challenge_status = recognize_faces(
                frame, known_encodings, known_names, challenge_status, penalty_list
            )
            
            # Display challenge status if active
            current_time = time.time()
            for name, status in list(challenge_status.items()):
                if status['active']:
                    cv2.putText(processed_frame, f"Challenge: {status['text']}", (20, 50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                    remaining = CHALLENGE_DURATION - (current_time - status['start_time'])
                    cv2.putText(processed_frame, f"Time left: {max(0, round(remaining, 1))}s", 
                               (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
                    
                    if check_challenge(status['text'], status['prev_frame'], processed_frame):
                        challenge_status[name]['active'] = False
                        print(f"{name} passed challenge!")
                    elif current_time - status['start_time'] > CHALLENGE_DURATION:
                        challenge_status[name]['active'] = False
                        penalty_list.append({'encoding': status['encoding'], 'time': current_time})
                        print(f"{name} failed challenge! Penalty active")
            
            # Cleanup expired penalties
            penalty_list = [p for p in penalty_list if current_time - p['time'] < PENALTY_DURATION]
            
            # Display
            cv2.imshow('Face Recognition (CNN)', processed_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        video_capture.release()
        cv2.destroyAllWindows()
        print("Face recognition stopped")

if __name__ == "__main__":
    main()

Loaded 133 known faces
Starting face recognition with CNN model (Threshold: 0.6)
Error: Could not read frame
Face recognition stopped


In [2]:
import face_recognition
import cv2
import pickle
import numpy as np
from scipy.spatial import distance as dist


# Configuration
THRESHOLD = 0.45
UNKNOWN_NAME = "Unknown"
EYE_AR_THRESH = 0.25  # Eye Aspect Ratio threshold (adjust as needed)
EYE_AR_CONSEC_FRAMES = 2  # Number of consecutive frames to confirm blink

def load_known_faces(filename):
    """Load known face encodings and names from file"""
    with open(filename, "rb") as f:
        known_face_encodings, known_face_names = pickle.load(f)
    return known_face_encodings, known_face_names

def eye_aspect_ratio(eye):
    """Compute Eye Aspect Ratio (EAR) to detect blinking"""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def recognize_faces(frame, known_encodings, known_names, counter=0, total=0):
    """Recognize faces with liveness check"""
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    # Initialize face landmarks detector (for eye blink detection)
    face_landmarks_list = face_recognition.face_landmarks(rgb_frame, face_locations)
    
    for (top, right, bottom, left), face_encoding, landmarks in zip(
        face_locations, face_encodings, face_landmarks_list
    ):
        # Check for blinking (liveness detection)
        left_eye = landmarks["left_eye"]
        right_eye = landmarks["right_eye"]
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        if ear < EYE_AR_THRESH:
            counter += 1
        else:
            if counter >= EYE_AR_CONSEC_FRAMES:
                total += 1
            counter = 0

        # Only recognize if blink detected (anti-spoofing)
        if total > 0:
            face_distances = face_recognition.face_distance(known_encodings, face_encoding)
            best_match_idx = np.argmin(face_distances)
            best_distance = face_distances[best_match_idx]
            name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
            color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)
        else:
            name = "Blink to verify"
            color = (255, 255, 0)  # Yellow (pending verification)

        # Draw face box and label
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    return frame, counter, total

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    video_capture = cv2.VideoCapture(0)
    counter = 0
    total = 0

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame, counter, total = recognize_faces(
            frame, known_encodings, known_names, counter, total
        )
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [1]:
import cv2
import pickle
import numpy as np
import face_recognition
from scipy.spatial import distance as dist
from skimage.feature import local_binary_pattern

# Configurations
THRESHOLD = 0.45
UNKNOWN_NAME = "Unknown"
EYE_AR_THRESH = 0.25  # Eye Aspect Ratio threshold
EYE_AR_CONSEC_FRAMES = 3  # Frames to confirm blink
REQUIRED_BLINKS = 2  # Minimum blinks to verify liveness
HEAD_MOVEMENT_THRESH = 15  # Pixel movement for head motion
TEXTURE_THRESH = 10  # LBP mean threshold (lower = smoother, likely fake)

def load_known_faces(filename):
    """Load known face encodings and names from file"""
    with open(filename, "rb") as f:
        known_face_encodings, known_face_names = pickle.load(f)
    return known_face_encodings, known_face_names

def eye_aspect_ratio(eye):
    """Compute Eye Aspect Ratio (EAR) for blink detection"""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def analyze_skin_texture(face_roi):
    """Check if skin texture is real using Local Binary Patterns (LBP)"""
    gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    texture_score = np.mean(lbp)
    return texture_score > TEXTURE_THRESH  # True if real skin

def challenge_user(frame, face_location):
    """Ask user to perform actions (blink/head movement)"""
    top, right, bottom, left = face_location
    cv2.putText(frame, "Blink or move your head!", (left, top - 10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
    return frame

def recognize_faces(frame, known_encodings, known_names, liveness_passed=False):
    """Recognize faces with liveness + texture checks"""
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    face_landmarks = face_recognition.face_landmarks(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding, landmarks in zip(
        face_locations, face_encodings, face_landmarks
    ):
        # Step 1: Liveness Check (Blink/Head Movement)
        if not liveness_passed:
            frame = challenge_user(frame, (top, right, bottom, left))
            left_ear = eye_aspect_ratio(landmarks["left_eye"])
            right_ear = eye_aspect_ratio(landmarks["right_eye"])
            ear = (left_ear + right_ear) / 2.0
            is_blinking = ear < EYE_AR_THRESH

            # Simulate head movement check (replace with actual tracking)
            has_moved = np.random.random() > 0.5  # Mock logic

            if is_blinking or has_moved:
                liveness_passed = True
                cv2.putText(frame, "Liveness passed!", (left, bottom + 40),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Step 2: Face Recognition (After Liveness)
        if liveness_passed:
            face_distances = face_recognition.face_distance(known_encodings, face_encoding)
            best_match_idx = np.argmin(face_distances)
            best_distance = face_distances[best_match_idx]
            name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
            color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)

            # Step 3: Skin Texture Check (After Recognition)
            face_roi = frame[top:bottom, left:right]
            is_real_skin = analyze_skin_texture(face_roi)
            if not is_real_skin:
                name = "FAKE (Texture)"
                color = (0, 0, 255)

            cv2.putText(frame, name, (left, bottom + 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

    return frame, liveness_passed

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    cap = cv2.VideoCapture(0)
    liveness_passed = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame, liveness_passed = recognize_faces(frame, known_encodings, known_names, liveness_passed)
        cv2.imshow("Anti-Spoof Face Recognition", frame)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [ ]:
import cv2
import numpy as np
import face_recognition
from skimage.feature import local_binary_pattern

# Configurations
THRESHOLD = 0.45
UNKNOWN_NAME = "Unknown"
TEXTURE_THRESH = 20  # Lowered threshold (adjust based on testing)
BLUR_THRESH = 50   # Discard blurred faces (variance threshold)

def load_known_faces(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

def analyze_skin_texture(face_roi):
    """Improved texture analysis with blur check."""
    gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    
    # 1. Check if face is too blurry (avoid false positives)
    blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
    if blur_score < BLUR_THRESH:
        return False  # Skip if face is blurry

    # 2. Analyze texture (LBP)
    lbp = local_binary_pattern(gray, 8, 1, method="uniform")
    texture_score = np.mean(lbp)
    return texture_score > TEXTURE_THRESH  # True if real skin

def recognize_faces(frame, known_encodings, known_names):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Step 1: Face recognition
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_idx = np.argmin(face_distances)
        best_distance = face_distances[best_match_idx]
        name = known_names[best_match_idx] if best_distance <= THRESHOLD else UNKNOWN_NAME
        color = (0, 255, 0) if name != UNKNOWN_NAME else (0, 0, 255)

        # Step 2: Skin texture check (only for recognized faces)
        if name != UNKNOWN_NAME:
            face_roi = frame[top:bottom, left:right]
            is_real_skin = analyze_skin_texture(face_roi)
            if not is_real_skin:
                name = "FAKE (Texture)"
                color = (0, 0, 255)

        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, bottom + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return frame

def main():
    known_encodings, known_names = load_known_faces("face_encodings.pkl")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = recognize_faces(frame, known_encodings, known_names)
        cv2.imshow("Face Recognition + Anti-Spoof", frame)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
    

NameError: name 'texture_score' is not defined